In [10]:
import os
import io
import sys
from dotenv import load_dotenv
import google.genai
from IPython.display import Markdown,display,update_display
import gradio as gr
import subprocess

In [11]:
load_dotenv()
GEMINI_API_KEY=os.getenv('GEMINI_API_KEY')

In [12]:
from google import genai
client = genai.Client(api_key=GEMINI_API_KEY)

In [37]:
system_message = "You are an assistant that reimplements Python code in high performance Java for an Windows 11 workstation.\
    Respond only with Java code; use-comments sparingly and do not provide any explanation or conclusion after the code  other than occasional comments.\
    the Java response needs to produce an identical output in the fastest possible time."

In [38]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in a Java class named Main with the fastest possible implementation that produces identical output in the least\
    time.Respond only in Java code; do not explain your work other than a few comments."
    user_prompt+="Pay attention to number types to ensure no int overflows. Remember to import all necessary Java packages such as java.util.*"
    user_prompt+=python
    return user_prompt

In [15]:
from google.genai.types import Content,Part
def messages_for(python):
    return [
        Content(
            role="user",
            parts=[Part(text=user_prompt_for(python))]
        )
    ]

In [16]:
def messages_for_llama(python):
    return[
        {"role":"system","content":system_message},
        {"role":"user","content":user_prompt_for(python)}
    ]

In [40]:
##Writing optimized java code to a file

def write_output(java):
    code = java.replace("```java","").replace("```","")
    cleaned = code[:code.rfind('}') + 1]
    with open("Main.java",'w') as f:
        f.write(cleaned)

In [18]:
import ollama

In [19]:
# import ollama

# def messages_for(python_code: str) -> str:
#     return f"Rewrite this Python code to be more efficient:\n\n{python_code}"

def optimize_llama(python):
    
    reply = ""

    # Streaming response from Ollama
    for chunk in ollama.chat(
        model="llama3.2",
        messages=messages_for_llama(python),
        stream=True
    ):
        print(chunk['message']['content'], end='', flush=True)
        reply += chunk['message']['content']

    write_output(reply)


In [20]:
from google.genai import types
def optimize_gemini(python):
    reply=""
    for chunk in client.models.generate_content_stream(
        model="gemini-2.0-flash",
        contents=messages_for(python),
        config=types.GenerateContentConfig(
            system_instruction = system_message
        )
    ):
        print(chunk.text,end='',flush=True)
        reply+=chunk.text
    write_output(reply)
        

In [21]:
pi="""
import time

def calculate(iterations,param1,param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000,4,1) * 4
end_time = time.time()

print(f"Result: {result : 12f}")
print(f"Execute Time: {(end_time - start_time):.6f} seconds")


"""

In [27]:
exec(pi)

Result:     3.141593
Execute Time: 13.504781 seconds


In [ ]:
optimize_gemini(pi)

In [ ]:
optimize_llama(pi)

In [39]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 67.706409 seconds


In [ ]:
optimize_gemini(python_hard)

In [23]:
from google.genai import types

def stream_gemini(python_code):
    reply = ""
    for chunk in client.models.generate_content_stream(
        model="gemini-2.0-flash",
        contents=messages_for(python_code),
        config=types.GenerateContentConfig(
            temperature=0.7,
            system_instruction=system_message
        )
    ):
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```java\n', '').replace('```', '')


In [24]:
def stream_ollama(python_code):
    reply = ""

    for chunk in ollama.chat(
        model="llama3.2",
        messages= messages_for_llama(python_code),
        stream=True
    ):
         content = chunk['message']['content']
         reply += content
         yield reply.replace('```java\n', '').replace('```', '')

In [25]:
def optimize(python, model):
    if model=="Gemini":
        result = stream_gemini(python)
    elif model=="Llama":
        result = stream_gemini(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far  

In [60]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:",lines=10,elem_id="py-code-box")
        java = gr.Textbox(label="Java code:",lines=10)
    with gr.Row():
        model = gr.Dropdown(["Gemini","Llama"],label="Select model",value="Gemini",elem_id="java-code-box")
        convert = gr.Button("Convert code")

    convert.click(optimize,inputs=[python,model],outputs=[java])
    
    ui.load(
        lambda: None,
        js="""
        () => {
            document.getElementById("py-code-box").setAttribute("spellcheck", "false");
            document.getElementById("java-code-box").setAttribute("spellcheck", "false");
        }
        """
    )
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [26]:
import textwrap
def execute_python(code):
    try:
        code = textwrap.dedent(code)
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [27]:

def execute_java(code):
    write_output(code)  # write the input code to Main.java
    compile_cmd = ["javac", "Main.java"]
    run_cmd = ["java", "Main"]

    try:
        # Compile the Java code
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)

        # Run the compiled Java code
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)

        return run_result.stdout

    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"


In [28]:
css = """
.python {
  background-color: #1c2b3a; /* Darker shade of Python blue */
}

.java {
  background-color: #002b00; /* Dark forest green for C++ */
}
 

"""

In [71]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to Java")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10,elem_id="py-code-box")
        java = gr.Textbox(label="Java code:", lines=10,elem_id="java-code-box")
    with gr.Row():
        model = gr.Dropdown(["Gemini", "Llama"], label="Select model", value="Gemini")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        java_run = gr.Button("Run Java")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        java_out = gr.TextArea(label="Java result:", elem_classes=["java"])

    convert.click(optimize, inputs=[python, model], outputs=[java])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    java_run.click(execute_java, inputs=[java], outputs=[java_out])
    
    ui.load(
        lambda: None,
        js="""
        () => {
            document.getElementById("py-code-box").setAttribute("spellcheck", "false");
            document.getElementById("java-code-box").setAttribute("spellcheck", "false");
        }
        """
    )
ui.launch(inbrowser=True)

In [41]:
# import ollama

# def messages_for(python_code: str) -> str:
#     return f"Rewrite this Python code to be more efficient:\n\n{python_code}"

def optimize_qwen(python):
    
    reply = ""

    # Streaming response from Ollama
    for chunk in ollama.chat(
        model="qwen2.5-coder:1.5b",
        messages=messages_for_llama(python),
        stream=True
    ):
        print(chunk['message']['content'], end='', flush=True)
        reply += chunk['message']['content']

    write_output(reply)


In [42]:
optimize_qwen(pi)

```java
import java.util.stream.LongStream;

public class Main {
    public static void main(String[] args) {
        long iterations = 100_000_000L;
        double param1 = 4.0;
        double param2 = 1.0;
        long start = System.currentTimeMillis();
        double result = calculate(iterations, param1, param2);
        double end = System.currentTimeMillis();

        System.out.printf("Result: %.12f\n", result);
        System.out.printf("Execute Time: %7.6f seconds\n", (end - start) / 1000.0);
    }

    public static double calculate(long iterations, double param1, double param2) {
        return LongStream.rangeClosed(1L, iterations)
                         .mapToDouble(i -> {
                             double j = i * param1 - param2;
                             double result = -1 / j;
                             j = i * param1 + param2;
                             return 1 / j;
                         }).average().getAsDouble();
    }
}
```
In this Java code, we've i

In [44]:
optimize_qwen(python_hard)

```java
import java.util.Random;

public class Main {
    public static void main(String[] args) {
        int n = 10000;
        int initialSeed = 42;
        int minVal = -10;
        int maxVal = 10;
        
        long startTime = System.currentTimeMillis();
        double result = totalMaxSubarraySum(n, initialSeed, minVal, maxVal);
        long endTime = System.currentTimeMillis();
        
        System.out.println("Total Maximum Subarray Sum (20 runs): " + result);
        System.out.println("Execution Time: " + ((endTime - startTime) / 1000.0) + " seconds");
    }
    
    public static double totalMaxSubarraySum(int n, int initialSeed, int minVal, int maxVal) {
        Random random = new Random(initialSeed);
        long[] randomNumbers = new long[n];
        
        for (int i = 0; i < n; i++) {
            randomNumbers[i] = random.nextInt(maxVal - minVal + 1) + minVal;
        }
        
        double totalSum = 0.0;
        int currentSum = 0;
        
        for (